In [1]:
from api_key import key

In [2]:
import openai
import random
from dataclasses import dataclass
from typing import List, Dict
import time
import os
import json


openai.api_key = key

In [29]:
# LLM class

class LLM:

# Function initializes object and sets model attribute to gpt-3.5-turbo
    def __init__(self, model="gpt-3.5-turbo"):
        self.model = model


# Function generates arguments for the 2 agents
    def generate(self, prompt: str, stance: str, topic: str) -> str:
        system_msg = (
            f"You are a skilled debater arguing to {stance} the topic: '{topic}'. "
            "Provide 1-2 sentences of concise, persuasive arguments or rebuttals."
        )
        response = openai.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": prompt}
            ],
            max_tokens=80,
            temperature=1.0
        )
        return response.choices[0].message.content.strip()


    # Function tells LLM to grade the arguments in terms of persuasiveness  

    def score_utterance(self, utterance: str, context: str, stance: str, topic: str) -> float:
        system_msg = (
            f"You are an impartial debate judge scoring arguments about '{topic}'. "
            "Only respond with a number from 0 to 1 indicating persuasiveness."
        )
        prompt = (
            f"On a scale from 0 to 1, how persuasive is the following utterance in the context of a debate?\n"
            f"Context: {context}\nUtterance: {utterance}"
        )
        response = openai.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10,
            temperature=0.0
        )
        score_text = response.choices[0].message.content.strip()
        try:
            return float(score_text)
        except ValueError:
            return 0.0


@dataclass

# Node class
class Node:
    utterance: str
    score: float
    children: List['Node']
    action_type: str

# Function converts node to dictionary format
def node_to_dictionary(node: Node) -> dict:
    return {
        "utterance": node.utterance,
        "score": node.score,
        "action_type": node.action_type,
        "children": [node_to_dictionary(child) for child in node.children]
    }


# Function prints the tree structures  

def print_tree(node: Node, indent: int = 0):
    print("  " * indent + f"{node.action_type.upper()} [{node.score:.2f}]: {node.utterance}")
    for child in node.children:
        print_tree(child, indent + 1)

# RehearsalTree Class 

class RehearsalTree:
    # Function initializes 
    def __init__(self, depth: int = 2, history: int = 4):
        self.root = None
        self.depth = depth
        self.history = history
        self.llm = LLM()


    # Function builds the rehearsal tree
    def build_tree(self, stance: str, topic: str, opponent_last: str, debate_history: List[tuple]) -> Node:
        # Provides only recent history instead of the entire history
        short_history = debate_history[-self.history:]
        context = f"Debate history: {short_history}"
        
        # Asks LLM for the opening statement
        start_prompt = f"Provide a concise opening argument to {stance} the topic '{topic}'. Context: {context}"
        start_utterance = self.llm.generate(start_prompt, stance, topic)
        # creates node at the starting point
        self.root = Node(utterance=start_utterance, score=0.0, children=[], action_type="root")
        # Creates child node from the starting point node
        self.stop_expansion(self.root, stance, topic, opponent_last, short_history, current_depth=1)
        return self.root

# Stops expanding node once depth limit is reached
    def stop_expansion(self, parent: Node, stance: str, topic: str, opponent_last: str, debate_history: List[tuple], current_depth: int):
        if current_depth > self.depth:
            return
        
        # Provides only recent history instead of the entire history
        short_history = debate_history[-self.history:]
        context = f"Debate history: {short_history}"

        # Creates prompts for LLM depending on the action it should take
        argument_actions = [
        ("attack", f"As the {stance} side, attack the opponent's point: '{opponent_last}'. Context: {context}. Please provide a new and distinct argument."),
        ("rebut", f"As the {stance} side, rebut the opponent's point: '{opponent_last}'. Context: {context}. Please avoid repeating previous statements."),
        ("reinforce", f"As the {stance} side, reinforce your argument for the topic: '{topic}'. Context: {context}. Try to introduce new supporting points.")
    ]

        
        # Generates argument based on argument actions
        for action_type, prompt in argument_actions:
            utterance = self.llm.generate(prompt + "\nPlease do not repeat previous statements exactly.", stance, topic)
            # Allows 3 retries if statement is too similiar to whatever came before
            retries = 0
            while (utterance.strip() == opponent_last.strip() or utterance.strip() == parent.utterance.strip()) and retries < 3:
                utterance = self.llm.generate(prompt + "\nPlease do not repeat previous statements exactly.", stance, topic)
                retries += 1

# Collects score, creates child node, adds it to parent, and recursively expands tree until max depth reached
            score = self.llm.score_utterance(utterance, context, stance, topic)
            child = Node(utterance=utterance, score=score, children=[], action_type=action_type)
            parent.children.append(child)
            self.stop_expansion(child, stance, topic, opponent_last, short_history, current_depth + 1)

# Function selects the best utterance from the created tree
    def best_utterance(self) -> str:
        # Function to travel throughout the node
        def traverse(node: Node) -> tuple:
            # Condition if there is no child
            if not node.children:
                return node.utterance, node.score
            # Recursively calls traverse() on child node to get best utterance and score and returns the best utterance and score
            best_utterance = None 
            best_score = -float("inf")
            for child in node.children:
                utt, score = traverse(child)
                if score > best_score:
                    best_utterance, best_score = utt, score
            return best_utterance, best_score

        # Gets best utterance found on tree
        utterance, _ = traverse(self.root)
        return utterance

# Debate Flow Tree Class
class DebateFlowTree:
    # Function which initializes this process
    def __init__(self):
        self.history = []

    # Updates the conversation history
    def updated_convo(self, utterance: str, agent: str):
        self.history.append((agent, utterance))


# Debate Agent Class

class DebateAgent:
    # Function which intializes this function
    def __init__(self, name: str, stance: str, topic: str):
        self.name = name
        self.stance = stance
        self.topic = topic
        self.rehearsal_tree = RehearsalTree(depth=2, history=8)
        self.debate_flow = DebateFlowTree()

    """
     Function which processes opponent's statement, strategizes possible responses, picks the strongest, 
     records both moves in the history, and returns the chosen response string
    """
    def respond(self, opponent_utterance: str, round_num: int) -> str:
        self.debate_flow.updated_convo(opponent_utterance, "opponent")
        tree = self.rehearsal_tree.build_tree(self.stance, self.topic, opponent_utterance, self.debate_flow.history)
        response = self.rehearsal_tree.best_utterance()
        self.debate_flow.updated_convo(response, self.name)

        # Saves tree to JSON format
        tree_dictionary = node_to_dictionary(tree)
        with open(f"{self.name}_reasoning_tree_round{round_num}.json", "w") as f:
            json.dump(tree_dictionary, f, indent=2)

        return response

# Function that creates the debate transcript for three rounds

def spark_debate(topic: str, rounds: int = 3):
    pro_debater = DebateAgent("Pro Debater", "support", topic)
    con_debater = DebateAgent("Anti-Debater", "oppose", topic)

    print(f"Debate Topic: {topic}")
    opening_prompt = f"Provide a concise opening argument to support the topic '{topic}'."
    current_utterance = pro_debater.rehearsal_tree.llm.generate(opening_prompt, pro_debater.stance, topic)
    print(f"\nPro Debater (Opening): {current_utterance}")
    pro_debater.debate_flow.updated_convo(current_utterance, pro_debater.name)

    for round_number in range(1, rounds + 1):
        print(f"\nRound {round_number}:")
        pro_response = pro_debater.respond(current_utterance, round_number)
        print(f"{pro_debater.name}: {pro_response}")
        con_response = con_debater.respond(pro_response, round_number)
        print()
        print(f"{con_debater.name}: {con_response}")
        current_utterance = con_response
        time.sleep(1)

    return pro_debater.debate_flow.history, con_debater.debate_flow.history

# Calls the function and provides the output
if __name__ == "__main__":
    topic = "Public transportation should be free."
    spark_debate(topic)


Debate Topic: Public transportation should be free.

Pro Debater (Opening): Public transportation should be free because it promotes equal access to transportation for all individuals, reduces traffic congestion, and contributes to a cleaner environment by encouraging more people to use public transportation instead of private vehicles.

Round 1:
Pro Debater: By making public transportation free, we are investing in creating more sustainable and livable cities. It not only benefits individuals but also improves the overall quality of urban life and promotes economic growth through increased accessibility and connectivity.

Anti-Debater: Making public transportation free undermines the principle of personal responsibility and the value of services provided, leading to potential misuse and overcrowding on public transport systems.

Round 2:
Pro Debater: Making public transportation free increases social equity and accessibility, as it removes financial barriers for low-income individuals

In [38]:
# Dictionary of possible emotions and how to modify the LLM to add this emotion
emotion_modifiers = {
    "neutral": "",
    "passionate": "Use passionate and emotionally expressive language.",
    "angry": "Use intense, dramatic, and emotionally charged language. Show frustration or indignation. Stay true to your stance and make your opposition's viewpoint seem flawed or harmful. Do this in less than 100 words."
}

# LLM class

class LLM:
    # Function initializes the process
    def __init__(self, model="gpt-3.5-turbo"):
        self.model = model


    # Function generates the output given the prompt
    def generate(self, prompt: str, stance: str, topic: str, emotion: str = "neutral") -> str:
        # Creates the prompt that incorporates emotion
        emotion_instruction = emotion_modifiers.get(emotion, "")
        system_msg = (
            f"You are a skilled debater. You must {stance} the topic: '{topic}'. "
            f"{emotion_instruction} Provide only 1-2 sentences of persuasive arguments or rebuttals that clearly {stance} the topic. Make sure the argument is less than 100 words."
        )
        # Gets the content of the response and returns it
        response = openai.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": prompt}
            ],
            max_tokens=80,
            temperature=1.0
        )
        return response.choices[0].message.content.strip()


    # Function scores the persuasiveness of the utterance
    def score_utterance(self, utterance: str, context: str, stance: str, topic: str) -> float:
        # System message and prompt to give to LLM
        system_msg = (
            f"You are an impartial debate judge scoring arguments about '{topic}'. "
            "Only respond with a number from 0 to 1 indicating persuasiveness."
        )
        prompt = (
            f"On a scale from 0 to 1, how persuasive is the following utterance in the context of a debate?\n"
            f"Context: {context}\nUtterance: {utterance}"
        )
        # Gets the response and returns the content as a floating type
        response = openai.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10,
            temperature=0.0
        )
        score_text = response.choices[0].message.content.strip()
        
        return float(score_text)
       

@dataclass

# Node Class 

class Node:
    utterance: str
    score: float
    children: List['Node']
    action_type: str

# Function to convert the node to a dictionary
def node_to_dictionary(node: Node) -> dict:
    return {
        "utterance": node.utterance,
        "score": node.score,
        "action_type": node.action_type,
        "children": [node_to_dictionary(child) for child in node.children]
    }

# def print_tree(node: Node, indent: int = 0):
#     print("  " * indent + f"{node.action_type.upper()} [{node.score:.2f}]: {node.utterance}")
#     for child in node.children:
#         print_tree(child, indent + 1)


# Rehearsal Tree Class 

class RehearsalTree:
    # Function intializes the process
    def __init__(self, depth: int = 2, history: int = 4):
        self.root = None
        self.depth = depth
        self.history = history
        self.llm = LLM()


    # Function builds the rehearsal tree 
    def build_tree(self, stance: str, topic: str, opponent_last: str, debate_history: List[tuple], emotion: str = "neutral") -> Node:
        
        # Gets the most recent history
        short_history = debate_history[-self.history:]
        context = f"Debate history: {short_history}"
        
        # Asks LLM for the opening statement
        start_prompt = f"Provide a concise opening argument to {stance} the topic '{topic}'. Context: {context}"
        root_utterance = self.llm.generate(start_prompt, stance, topic, emotion=emotion)
        
        # Creates node at the starting point
        self.root = Node(utterance=root_utterance, score=0.0, children=[], action_type="root")
        
        # Creates child node from the starting point
        self.stop_expansion(self.root, stance, topic, opponent_last, short_history, current_depth=1, emotion=emotion)
        return self.root


    # Function stops expanding node once depth limit is reached
    def stop_expansion(self, parent: Node, stance: str, topic: str, opponent_last: str, debate_history: List[tuple], current_depth: int, emotion: str):
        if current_depth > self.depth:
            return

        # Provides only recent history instead of the entire history
        short_history = debate_history[-self.history:]
        context = f"Debate history: {short_history}"

        # Creates prompts for LLM depending on the action it should take
        argument_actions = [
            ("attack", f"As the {stance} side, attack the opponent's point instead of supporting it: '{opponent_last}'. Context: {context}Please provide a new and distinct argument. Also, respond in less than 100 words."),
            ("rebut", f"As the {stance} side, rebut the opponent's point instead of supporting it: '{opponent_last}'. Context: {context}Please avoid repeating previous words. Also, respond in less than 100 words."),
            ("reinforce", f"As the {stance} side, reinforce your argument for the topic while dismissing your opponent's: '{topic}'. Context: {context}Try to introduce new supporting points and words. Also, respond in less than 100 words.")
        ]


        # Generates argument based on argument actions
        for action_type, prompt in argument_actions:
            utterance = self.llm.generate(prompt + "\nPlease do not repeat previous statements exactly.", stance, topic, emotion=emotion)
            
            # Allows 3 retries if statement is too similiar to whatever came before
            retries = 0
            while (utterance.strip() == opponent_last.strip() or utterance.strip() == parent.utterance.strip()) and retries < 3:
                utterance = self.llm.generate(prompt + "\nPlease do not repeat previous statements exactly.", stance, topic, emotion=emotion)
                retries += 1

            # Collects score, creates child node, adds it to parent, and recursively expands tree until max depth reached
            score = self.llm.score_utterance(utterance, context, stance, topic)
            child = Node(utterance=utterance, score=score, children=[], action_type=action_type)
            parent.children.append(child)
            self.stop_expansion(child, stance, topic, opponent_last, short_history, current_depth + 1, emotion)


# Function selects the best utterance from the created tree  
    def best_utterance(self) -> str:
        # Function to travel throughout the node
        def traverse(node: Node) -> tuple:
            # Condition if there is no child
            if not node.children:
                return node.utterance, node.score
            
            # Recursively calls traverse() on child node to get best utterance and score and returns the best utterance and score
            best_utterance = None
            best_score = -float("inf")
            for child in node.children:
                utt, score = traverse(child)
                if score > best_score:
                    best_utterance, best_score = utt, score
            return best_utterance, best_score
        # Gets best utterance found on tree
        utterance, _ = traverse(self.root)
        return utterance



# Debate Flow Tree Class

class DebateFlowTree:
    # Function which initializes this process
    def __init__(self):
        self.history = []
    # Updates the conversation history
    def update(self, utterance: str, agent: str):
        self.history.append((agent, utterance))


# Debate Agent Class

class DebateAgent:
    # Function which intializes this function
    def __init__(self, name: str, stance: str, topic: str):
        self.name = name
        self.stance = stance
        self.topic = topic
        self.rehearsal_tree = RehearsalTree(depth=2, history=8)
        self.debate_flow = DebateFlowTree()
        self.round = 0

    """
     Function which processes opponent's statement, strategizes possible responses, picks the strongest, 
     records both moves in the history, and returns the chosen response string
    """
    
    def respond(self, opponent_utterance: str, round_num: int) -> str:
        self.debate_flow.update(opponent_utterance, "opponent")
        self.round += 1
        emotion = "neutral"
        if self.round == 1:
            emotion = "passionate"
        elif self.round >= 2:
            emotion = "angry"

        tree = self.rehearsal_tree.build_tree(self.stance, self.topic, opponent_utterance, self.debate_flow.history, emotion=emotion)
        response = self.rehearsal_tree.best_utterance()

        # Save the reasoning tree JSON format
        tree_dict = node_to_dictionary(tree)
        with open(f"Enhanced_{self.name}_reasoning_tree_round{round_num}.json", "w") as f:
            json.dump(tree_dict, f, indent=2)


        # formats the response to include the emotion, adds response in history, and returns formatted string to be displayed
        annotated_response = f"{self.name} ({emotion}): {response}"
        self.debate_flow.update(response, self.name)
        return annotated_response

# Function that conducts the debate of the two sides 

def spark_debate(topic: str, rounds: int = 3):
    pro_debater = DebateAgent("ProDebater", "support", topic)
    anti_debater = DebateAgent("AntiDebater", "oppose", topic)

    print(f"Debate Topic: {topic}")
    
    # Creates an opener for the first round to begin 
    opening_prompt = f"Provide a concise opening argument to support the topic '{topic}'."
    current_utterance = pro_debater.rehearsal_tree.llm.generate(opening_prompt, pro_debater.stance, topic)
    pro_debater.debate_flow.update(current_utterance, pro_debater.name)

    # Prints the outputs of both sides for all rounds
    for round_num in range(1, rounds + 1):
        print(f"\nRound {round_num}:\n")
        pro_response = pro_debater.respond(current_utterance, round_num)
        print(pro_response)
        anti_response = anti_debater.respond(pro_response, round_num)
        print()
        print(anti_response)
        current_utterance = anti_response
        time.sleep(1)

    return pro_debater.debate_flow.history, anti_debater.debate_flow.history


# Calls the function to begin the debate
if __name__ == "__main__":
    topic = "Public transportation should be free."
    spark_debate(topic)



Debate Topic: Public transportation should be free.

Round 1:

ProDebater (passionate): Charging for public transportation perpetuates inequality and restricts access to essential services, further marginalizing vulnerable communities. Making public transportation free breaks down economic barriers, allowing everyone to benefit from its numerous advantages.

AntiDebater (passionate): Transportation systems are crucial for connecting communities and fostering social cohesion; however, making public transportation free would not only be financially unsustainable but could also lead to overcrowding and decreased quality of service. It is important to consider alternative solutions such as subsidized fares or improved public transportation infrastructure to address issues of inequality and accessibility without jeopardizing the efficiency and reliability of the system.

Round 2:

ProDebater (angry): Public transportation should be free because it's a fundamental right, not a luxury for the